In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
#Load the ANN trained model, and all pickle files

model=load_model('model.h5')

with open("label_encoder.pkl","rb") as file:
    label_encoder_geo=pickle.load(file)

with open("scaler.pkl","rb") as file:
    scaler=pickle.load(file)

with open("one_hot_encoded.pkl","rb") as file:
    one_hot=pickle.load(file)


In [4]:
#Example New Test Data

input_data={
    "CreditScore": 600,
    "Geography": "France",
    "Gender": "Male",
    "Age": 40,
    "Tenure": 3,
    "Balance": 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000
}

In [5]:
df=pd.DataFrame([input_data])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [6]:
df['Gender']=label_encoder_geo.transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [8]:
encoded=one_hot.transform(df[['Geography']])
encoded

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1 stored elements and shape (1, 3)>

In [9]:
geo_encoded=pd.DataFrame(encoded.toarray(),columns=one_hot.get_feature_names_out(["Geography"]))

In [10]:
geo_encoded

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [11]:
data=pd.concat([df.drop("Geography",axis=1),geo_encoded],axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [12]:
data_scaled=scaler.transform(data)
data_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [13]:
final_predict=model.predict(data_scaled)
final_predict

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


array([[0.02746584]], dtype=float32)

In [14]:
predcition_proba=final_predict[0][0]

In [15]:
if predcition_proba>0.5:
    print("Customer likley to churn")
else:
    print("Customer doesn't churn")

Customer doesn't churn
